# Deploy cohere-command-r Model Package from AWS Marketplace 

**MAKE SURE TO UPDATE ALL LINKS TO POINT TO COMMAND-R**
Cohere builds a collection of Large Language Models (LLMs) trained on a massive corpus of curated web data. Powering these models, our infrastructure enables our product to be deployed for a wide range of use cases. The use cases we power include generation (copy writing, etc), summarization, classification, content moderation, information extraction, semantic search, and contextual entity extraction

This sample notebook shows you how to deploy [cohere-command-r](https://aws.amazon.com/marketplace/pp/TODO_CHANGE_TO_LISTING) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

> cohere-command model package support SageMaker Realtime Inference but not SageMaker Batch Transform.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [cohere-command-r](https://aws.amazon.com/marketplace/pp/prodview-n44fbeuycwldi). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Streaming Chat](#E.-Streaming-Chat)
   6. [Chat with documents (RAG)](#F.-Chat-with-documets-(RAG))
   7. [Generate search queries](#G.-generate-search-queries)
   8. [Tool inputs](#H.-Tool-inputs)
   9. [Tool results](#I.-Tool-results)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [cohere-command-r](https://aws.amazon.com/marketplace/pp/prodview-n44fbeuycwldi)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
!pip install --upgrade cohere-aws
# if you upgrade the package, you need to restart the kernel

from cohere_aws import Client
import boto3

  Attempting uninstall: cohere-aws
    Found existing installation: cohere-aws 0.8.10
    Uninstalling cohere-aws-0.8.10:
      Successfully uninstalled cohere-aws-0.8.10

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lfayoux/Library/Application Support/sagemaker/config.yaml


In [ ]:
cohere_package = "cohere-command-r-8k-v1-0"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{cohere_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{cohere_package}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{cohere_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{cohere_package}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{cohere_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{cohere_package}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{cohere_package}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{cohere_package}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{cohere_package}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{cohere_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{cohere_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{cohere_package}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{cohere_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{cohere_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{cohere_package}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{cohere_package}",
}

region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Create an endpoint

In [ ]:
co = Client(region_name=region)
co.create_endpoint(arn=model_package_arn, endpoint_name="cohere-command-r", instance_type="ml.p4de.24xlarge", n_instances=1)

# If the endpoint is already created, you just need to connect to it
# co.connect_to_endpoint(endpoint_name="cohere-command-r")

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
message = "Write a LinkedIn post about starting a career in tech:"

### C. Perform real-time inference

In [ ]:
response = co.chat(message=message, stream=False)

### D. Visualize output

In [ ]:
print(response)

### E. Streaming Chat

In [ ]:
message = "Write a LinkedIn post about starting a career in tech:"

response = co.chat(message=message, stream=True)

# stream events back
for res in response:
    print(res)

### F. Chat with documents (RAG)

In [ ]:
message="How deep in the Mariana Trench"
documents = [
    {
       "id": "national_geographic_everest",
       "title": "Height of Mount Everest",
       "snippet": "The height of Mount Everest is 29,035 feet",
       "url": "https://education.nationalgeographic.org/resource/mount-everest/",
    },
    {
        "id": "national_geographic_mariana",
        "title": "Depth of the Mariana Trench",
        "snippet": "The depth of the Mariana Trench is 36,070 feet",
        "url": "https://www.nationalgeographic.org/activity/mariana-trench-deepest-place-earth",
    }
]

response = co.generate(prompt=message, documents=documents, stream=False)

print(response)

### G. Generate search queries

In [ ]:
message="What is the height of Mount Everest?"

response = co.chat(message=message, search_queries_only=True, stream=False)

print(response)

### H. Tool inputs

In [ ]:
message="What were sales like on October 31st?"
tools=[
    {
		"name": "sales_database",
        "description": "Connects to a database about sales volumes",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data from this day, formatted as YYYY-MM-DD",
                "type": "str",
                "required": True
            }
        }
    }
]

response = co.chat(message=message, tools=tools, stream=False)

print(response)

### I. Tool results

In [ ]:
message="What were sales like on October 31st?"
tools=[
    {
		"name": "sales_database",
        "description": "Connects to a database about sales volumes",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data from this day, formatted as YYYY-MM-DD",
                "type": "str",
                "required": True
            }
        }
    }
]
tool_results=[
    {
        "call": {
            "name": "sales_database",
            "parameters": {
                "day": "2023-04-08"
            }
        },
        "outputs": [{"number_of_sales": 120, "total_revenue": 48500, "day": "2023-04-08"}]
    }
]

response = co.chat(message=message, tools=tools, tool_results=tool_results, stream=False)

print(response)

## 4. Clean-up

### A. Delete the endpoint

In [ ]:
co.delete_endpoint()
co.close()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

